In [1]:
# import necessary library
import pandas as pd
import numpy as np
import random

## 1. Hello, Data 
* Load 1000 sales record file and just store 500 data set in data frame
* show only first 3 row

* reference
   - 1000 Sales Records : https://excelbianalytics.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

In [2]:
# load data
raw_primary = pd.read_csv("data/1000 Sales Records.csv")

# declare 500 data and just store rows
primary_500 = raw_primary.head(500)

# show first 3 rows
primary_500.head(3)



,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,10/31/2014,8446,437.20,263.33,3692591.20,2224085.18,1468506.02
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,12/8/2011,3018,154.06,90.93,464953.08,274426.74,190526.34
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,12/9/2016,1517,255.28,159.42,387259.76,241840.14,145419.62


## 1-1 Load Secondary Metadata File 
* load secondary file
- Coupon reference: https://www.kaggle.com/datasets/rishikumarrajvansh/marketing-insights-for-e-commerce-company?select=Discount_Coupon.csv
- City reference: https://www.kaggle.com/datasets/dataanalyst001/all-capital-cities-in-the-world


In [3]:
raw_city_data = pd.read_csv("data/all capital cities in the world.csv")
raw_coupon_data = pd.read_csv("data/Discount_Coupon.csv")

## 2. Pick the Right Container
* Dictionnary는 key와 value로 구성되어있다. 그렇기 때문에 key릅이용해 값에 빠르게 접급할수있지만 중복된 key를 사용할수없다.
* Set은 unordere한 unique한 값들의 collection이기때문에 중복될수없다.
* namedtupled은 불변한 콜렉션이다. 즉 수정과 삭제 가 불가하다. 하지만 preserve order해주고 index로 접근 가능하다.

이번 과제에서는 dataframe과 주로 dictionary와 set을 사용할것이다. 


In [4]:
# check data info/description
primary_500.info()
#primary_500.describe()
#primary_500.isnull().sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Region          500 non-null    object 
 1   Country         500 non-null    object 
 2   Item Type       500 non-null    object 
 3   Sales Channel   500 non-null    object 
 4   Order Priority  500 non-null    object 
 5   Order Date      500 non-null    object 
 6   Order ID        500 non-null    int64  
 7   Ship Date       500 non-null    object 
 8   Units Sold      500 non-null    int64  
 9   Unit Price      500 non-null    float64
 10  Unit Cost       500 non-null    float64
 11  Total Revenue   500 non-null    float64
 12  Total Cost      500 non-null    float64
 13  Total Profit    500 non-null    float64
dtypes: float64(5), int64(2), object(7)
memory usage: 54.8+ KB


## 3. Implement Functions and  Data structure

In [22]:
class Lab2Kihoon:
    
    def __init__(self):
        pass

    def standardize(data):
        
        # STEP1: rename columns to lowercase and remove space start and end for consitent
        data.columns = data.columns.str.lower().str.strip()

        # STEP2: replace space to _
        data.columns = data.columns.str.replace(" ","_")
        
        # STEP3: rename coulmns(Must contain)
        # example: item_type to product

            # add if statement  to test many times
        if "item_type" in data.columns:
            data["product"] = data["item_type"]
        
        if "unit_price" in data.columns: 
            data["price"] = data["unit_price"]
        
        if "units_sold" in data.columns: 
            data["quantity"] = data["units_sold"]
        
        if "order_date" in data.columns: 
            data["date"] = data["order_date"]

        
        # Step 4: type casting

        data["date"] = pd.to_datetime(data["date"])
        data["ship_date"] = pd.to_datetime(data["ship_date"])
        data["product"] = data["product"].astype("string")
        data["order_id"] = data["order_id"].astype("string")
        data["price"] = pd.to_numeric(data["price"])
        data["price"] = pd.to_numeric(data["price"])

        
        return data

    def clean(data):

        # print raw length
        print(f"Before row count: {len(data)}")

        
        # Step 5: coupon_code 같은경우에는 업을수도 있기때문에 NaN으로 채운다.
        data["coupon_code"] = data["coupon_code"].fillna("NONE")
        #현재 primary데이터에 city data에 없는 국가가 있기 때문에 null이 존재한다 그래서 NONE으로채운다
        data["shipping_city"] = data["shipping_city"].fillna("NONE")
        # Step 6: 필수컬럼 비어있는거 Drop 선택사항. 현재 데이터는 Null 값이 없다.
        data = data.dropna(subset=["date", "price", "quantity"])
        
        # Step 7: drop duplicate row
        data = data.drop_duplicates()

        # Step 8: strip extra whitespace in string columns
        data["region"] = data["region"].str.strip()
        data["country"] = data["country"].str.strip()
        data["item_type"] = data["item_type"].str.strip()
        data["sales_channel"] = data["sales_channel"].str.strip()
        data["order_priority"] = data["order_priority"].str.strip()
        data["order_id"] = data["order_id"].str.strip()
        data["product"] = data["product"].str.strip()

        print(f"After row count: {len(data)}")
        return data 

        
    def total(data):
        total = (data["price"] * data["price"]).sum()

    def total_with_coupon(data):
        total = (data["price"] * data["quantity"] * (1 - data["discount_percent"]/100)).sum()

    

In [7]:
# making customer_id: combination of year +"CUST"+ order_id 

def add_customer_id(data):
    year = pd.to_datetime(data["date"]).dt.year.astype(str)
    order_id = data["order_id"].astype(str)
    
    data["customer_id"] = (year+"CUST"+order_id)

    return data


## STEP8(Partial). Transformations

- 현재 필수적 컬럼인 shipping_city, coupon_code, customer_id가 없어 부분적으로 Transformation 및 다른작업 실시.


In [12]:
def add_shipping_city(primary_500, raw_city_data):

    # standilized value
    primary_500["country_key"] = (primary_500["country"].astype(str).str.strip().str.lower())


    needed_city_data = raw_city_data[["Country","Capital City"]]

    # rename column 
    needed_city_data = (needed_city_data[["Country", "Capital City"]].rename(columns={
        "Country": "sd_country",
        "Capital City": "sd_city"
        }))

    # standilized value
    needed_city_data["country_key"] = (needed_city_data["sd_country"].astype(str).str.strip().str.lower())

    # joim two dataset
    primary_500["sd_country"] = primary_500["country"]
    primary_500 = primary_500.merge(needed_city_data[["country_key", "sd_city"]], on="country_key", how="left")


    primary_500 = primary_500.drop(columns=["country_key","sd_country"])
    primary_500 = primary_500.rename(columns={
        "sd_city": "shipping_city"
    })
    return primary_500



In [9]:
def add_coupon_id(primary_500,raw_coupon_data):
    
    primary_500["order_month"] = primary_500["date"].dt.strftime("%b")

    needed_coupon_data = raw_coupon_data[["Month","Coupon_Code"]]

    # rename column 
    needed_coupon_data.columns = needed_coupon_data.columns.str.lower().str.strip()
    needed_coupon_data.columns = needed_coupon_data.columns.str.replace(" ","_")

    code_dict = needed_coupon_data.groupby("month")["coupon_code"].apply(list).to_dict()

    random_code = []

    for month_key in primary_500["order_month"]:
        coupon_code_list = code_dict.get(month_key)

        select = random.choice(coupon_code_list)

        random_code.append(select)

    primary_500["coupon_code"] = random_code

    return primary_500

In [13]:

primary_500 = Lab2Kihoon.standardize(primary_500)

primary_500 = add_customer_id(primary_500)

primary_500 = add_coupon_id(primary_500,raw_coupon_data)

primary_500 = add_shipping_city(primary_500, raw_city_data)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19752\2396365005.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["product"] = data["item_type"]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19752\2396365005.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["price"] = data["unit_price"]
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_19752\2396365005.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [ ]:
primary_500

## 4. Bulk Loaded
* product 값을 키로 가지고 product별 total profit를 값으로 dictionary로 만들었다. 쉽게 product별 total profit를 볼수있을것이다.

In [74]:
profit_by_product__dict = primary_500.groupby("product")["total_profit"].sum().to_dict()

profit_by_product__dict

{'Baby Food': 24471524.24,
 'Beverages': 3717621.36,
 'Cereal': 19037016.51,
 'Clothes': 13516485.12,
 'Cosmetics': 36463320.92,
 'Fruits': 387887.09,
 'Household': 34060166.68,
 'Meat': 11766440.4,
 'Office Supplies': 33954306.25,
 'Personal Care': 6171175.36,
 'Snacks': 9934408.379999999,
 'Vegetables': 12445700.72}

## 5. Quick Profiling
* price의 Min/Mean/Max를 구한다.
* set을 사용해 shipping city count를한다.

In [ ]:

# min/max/mean
min_price = primary_500["price"].min()
mean_price = primary_500["price"].mean()
max_price = primary_500["price"].max()

print(f"1. min price : {min_price}")
print(f"2. mean price : {mean_price}")
print(f"3. max price : {max_price}")


# unique city
cities_set = set(primary_500["shipping_city"])
unique_cities = len(cities_set)
print(f"4. unique cities count : {unique_cities}")



1. min price : 9.33
2. mean price : 274.29506
3. max price : 668.27
4. unique cities count : 166


## 6.  Spot the Grime
* 개인적으로 order_id는 고유한 식별자로 중복되면 안된다고 생각하기때문에 중복체크
* 금액관련한 value들은 0 이상이여한다고 생각해서 음수체크
* shipping city의 경우 null값이 존재하면 안된다고 생각하기때문에 null 체크

In [19]:
# order_id 중복체크
duplicated_order_id = primary_500["order_id"].duplicated().sum()

print(f"1. The number of duplicated order_id: {duplicated_order_id}")
# negative value check in price column
negative_price = (primary_500["price"]<0).sum()
negative_unit_cost = (primary_500["unit_cost"]<0).sum()
negative_total_cost = (primary_500["total_cost"]<0).sum()
negative_total_profit = (primary_500["total_profit"]<0).sum()
negative_total_revenue = (primary_500["total_revenue"]<0).sum()

print(f"2. The number of negative value in price: {negative_price}")
print(f"2. The number of negative value in unit_cost: {negative_unit_cost}")
print(f"2. The number of negative value in total_cost: {negative_total_cost}")
print(f"2. The number of negative value in total_profit: {negative_total_profit}")
print(f"2. The number of negative value in total_revenue: {negative_total_revenue}")



# Null check in shipping city

null_city = primary_500["shipping_city"].isnull().sum()

print(f"3. The number of null value in shipping_city: {null_city}")



1. The number of duplicated order_id: 0
2. The number of negative value in price: 0
2. The number of negative value in unit_cost: 0
2. The number of negative value in total_cost: 0
2. The number of negative value in total_profit: 0
2. The number of negative value in total_revenue: 0
3. The number of null value in shipping_city: 17


## 7. Cleaning Rules

In [23]:
Lab2Kihoon.clean(primary_500)

Before row count: 500
After row count: 500


,region,country,item_type,sales_channel,order_priority,order_date,order_id,ship_date,units_sold,unit_price,...,total_cost,total_profit,product,price,quantity,date,customer_id,order_month,coupon_code,shipping_city
0,Middle East and North Africa,Libya,Cosmetics,Offline,M,10/18/2014,686800706,2014-10-31,8446,437.20,...,2224085.18,1468506.02,Cosmetics,437.20,8446,2014-10-18,2014CUST686800706,Oct,AND10,Tripoli
1,North America,Canada,Vegetables,Online,M,11/7/2011,185941302,2011-12-08,3018,154.06,...,274426.74,190526.34,Vegetables,154.06,3018,2011-11-07,2011CUST185941302,Nov,OFF20,Ottawa
2,Middle East and North Africa,Libya,Baby Food,Offline,C,10/31/2016,246222341,2016-12-09,1517,255.28,...,241840.14,145419.62,Baby Food,255.28,1517,2016-10-31,2016CUST246222341,Oct,NCA10,Tripoli
3,Asia,Japan,Cereal,Offline,C,4/10/2010,161442649,2010-05-12,3322,205.70,...,389039.42,294295.98,Cereal,205.70,3322,2010-04-10,2010CUST161442649,Apr,HOU10,Tokyo
4,Sub-Saharan Africa,Chad,Fruits,Offline,H,8/16/2011,645713555,2011-08-31,9845,9.33,...,68127.40,23726.45,Fruits,9.33,9845,2011-08-16,2011CUST645713555,Aug,GC20,N'Djamena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Middle East and North Africa,Algeria,Meat,Offline,M,9/2/2011,183022201,2011-10-15,9191,421.89,...,3351865.79,525725.20,Meat,421.89,9191,2011-09-02,2011CUST183022201,Sep,NJ30,Algiers
496,Europe,Italy,Personal Care,Online,L,3/21/2011,127589738,2011-04-02,5494,81.73,...,311344.98,137679.64,Personal Care,81.73,5494,2011-03-21,2011CUST127589738,Mar,WEMP30,Rome
497,Europe,Russia,Fruits,Offline,L,1/8/2011,221530139,2011-01-26,4546,9.33,...,31458.32,10955.86,Fruits,9.33,4546,2011-01-08,2011CUST221530139,Jan,WEMP10,Moscow
498,Central America and the Caribbean,Antigua and Barbuda,Office Supplies,Offline,M,2/22/2015,363329732,2015-02-22,6197,651.21,...,3253177.12,782371.25,Office Supplies,651.21,6197,2015-02-22,2015CUST363329732,Feb,ACC20,Saint John's


## 8. Transformations
* coupon_code는 현재 string인데 숫자값만 extract 하여 numeric_discount 컬럼으로 transform 한다.
* 모든 국가는 Title case여야한다.

In [27]:
# 1. parse string to numeric
primary_500["numeric_discount"] = pd.to_numeric(primary_500["coupon_code"].str.extract(r"(\d+)")[0])


# 2. Uppercase
primary_500["country"] = primary_500["country"].str.title()

primary_500[["country","coupon_code","numeric_discount"]]

,country,coupon_code,numeric_discount
0,Libya,AND10,10
1,Canada,OFF20,20
2,Libya,NCA10,10
3,Japan,HOU10,10
4,Chad,GC20,20
...,...,...,...
495,Algeria,NJ30,30
496,Italy,WEMP30,30
497,Russia,WEMP10,10
498,Antigua And Barbuda,ACC20,20


## 9. Feature Engineering
* date 값과 오늘 기준의 값을 계산하여, 거래가 얼마나 지났는지 계산한다.

In [31]:
today = pd.to_datetime(pd.Timestamp.today().date())
primary_500["days_since_purchase"] = (today - primary_500["date"]).dt.days


primary_500[["date","days_since_purchase"]]

,date,days_since_purchase
0,2014-10-18,4000
1,2011-11-07,5076
2,2016-10-31,3256
3,2010-04-10,5652
4,2011-08-16,5159
...,...,...
495,2011-09-02,5142
496,2011-03-21,5307
497,2011-01-08,5379
498,2015-02-22,3873


## 10. Mini-Aggregation
* panas의 gorupby를 사용하여 shipping_city별 total_revenue를 보여준다.

In [32]:
revenue_by_shipping_city = primary_500.groupby("shipping_city")["total_revenue"].sum()

revenue_by_shipping_city

shipping_city
Abu Dhabi       665128.18
Abuja           131748.76
Accra          8138014.18
Addis Ababa     281251.62
Algiers        5752785.28
                  ...    
Wellington     6533529.54
Windhoek        344052.60
Yaounde          95209.92
Yerevan        7103563.75
Zagreb          879495.94
Name: total_revenue, Length: 166, dtype: float64

## 11. Serialization Checkpoint
* 클린되고 step8,9에서 추가되어진 컬럼들을 포함한 final cleaned 데이터를 csv와 json형식으로 저장한다.

In [ ]:
# serialize to csv
primary_500.to_csv("output/final_clean_date.csv")

# serialzie to json(record = one row is one object)
primary_500.to_json("output/final_clean_date.json", orient="records")